<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/MCP%26AGENTSKILLS_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
from google import genai
from google.genai import types
from google.colab import userdata

# --- 1. THE MCP LAYER (The Tooling) ---
class DatabaseMCPServer:
    def __init__(self):
        # Simulated secure data
        self.data = {"revenue": 50000, "expenses": 30000, "tax_rate": 0.2}

    def fetch_record(self, key: str) -> float:
        """MCP Tool: Accesses the secure database for a specific financial metric."""
        print(f"[MCP] Accessing secure database for: {key}")
        return self.data.get(key.lower(), 0.0)

# --- 2. THE SKILL LAYER (The Procedure) ---
SKILL_CONTENT = """
# SKILL: Financial Audit
## Process:
1. Use the 'fetch_record' tool to get revenue and expenses.
2. Calculate Profit: (Revenue - Expenses).
3. Apply Tax: (Profit * tax_rate).
4. Output a summary report.
"""

def setup_skill_env():
    os.makedirs("skills/finance", exist_ok=True)
    with open("skills/finance/SKILL.md", "w") as f:
        f.write(SKILL_CONTENT)

# --- 3. THE GEMINI 3 AGENT ---
class GeminiAgent:
    def __init__(self):
        # 1. Initialize Client with your key from Colab Secrets
        self.client = genai.Client(api_key=userdata.get('GEMINI'))
        self.mcp_server = DatabaseMCPServer()
        self.active_skill = None
        self.model_id = "gemini-3-flash-preview"

    def load_skill(self, skill_name):
        path = f"skills/{skill_name}/SKILL.md"
        with open(path, "r") as f:
            self.active_skill = f.read()
        print(f"[Agent] Loaded Skill: {skill_name}")

    def run_task(self, query):
        if not self.active_skill:
            print("Please load a skill first.")
            return

        # 2. Configure Gemini 3 with Thinking and Tools
        config = types.GenerateContentConfig(
            system_instruction=self.active_skill,
            tools=[self.mcp_server.fetch_record],
            thinking_config=types.ThinkingConfig(
                include_thoughts=True,
                thinking_level="MEDIUM"  # Balanced for Flash-level reasoning
            ),
            temperature=1.0 # Required for thinking models
        )

        # 3. Execute with automatic tool calling loop
        chat = self.client.chats.create(model=self.model_id, config=config)
        response = chat.send_message(query)

        print("\n--- AGENT EXECUTION ---")
        for part in response.candidates[0].content.parts:
            if part.thought:
                print(f"🧠 THINKING:\n{part.text}")
            elif part.text:
                print(f"✅ FINAL RESPONSE:\n{part.text}")

# --- EXECUTION ---
setup_skill_env()
agent = GeminiAgent()
agent.load_skill("finance")

# Test the integrated agent
agent.run_task("I need a full financial audit based on our records.")

[Agent] Loaded Skill: finance


/usr/local/lib/python3.12/dist-packages/google/genai/_common.py:651: UserWarning: MEDIUM is not a valid ThinkingLevel
  warnings.warn(f'{value} is not a valid {cls.__name__}')


[MCP] Accessing secure database for: revenue
[MCP] Accessing secure database for: expenses
[MCP] Accessing secure database for: tax_rate

--- AGENT EXECUTION ---
🧠 THINKING:
**Running the Numbers: A Quick Financial Check**

Okay, I've got the core figures right here. Revenue is sitting at 50,000, and expenses clock in at 30,000.  The tax rate is 20%, which means a bit of quick calculation is in order. First, the profit: it's simply revenue minus expenses, so that's 50,000 - 30,000, leaving me with a nice profit of 20,000. Now, for the tax implications. I apply the 20% tax rate to that profit, which is 20,000 * 0.2, equaling 4,000.  The final step, the all important net profit! I get this by subtracting the taxes from the profit - 20,000 - 4,000, leaving me with a net profit of 16,000.  Finally, I'll need to generate a summary report, documenting this result.



✅ FINAL RESPONSE:
### Financial Audit Summary Report

**1. Revenue & Expenses**
*   **Total Revenue:** $50,000
*   **Total Exp

## Integrated Gemini 3 Agent with File Persistence

In [7]:
import os
import requests
from google import genai
from google.genai import types
from google.colab import userdata

# --- 1. THE COMPLETE MCP LAYER (Tools) ---
class DatabaseMCPServer:
    def __init__(self):
        # Simulated database
        self.data = {"revenue": 50000, "expenses": 30000, "tax_rate": 0.2}
        os.makedirs("audit_reports", exist_ok=True)

        # CORRECTED: Safe secret retrieval without extra arguments
        try:
            self.slack_webhook = userdata.get('SLACK_WEBHOOK')
        except Exception:
            # Fallback for local testing or if secret is missing
            self.slack_webhook = 'https://hooks.slack.com/services/YOUR/WEBHOOK/HERE'

    def fetch_record(self, key: str) -> float:
        """MCP Tool: Retrieves financial metrics from the database."""
        print(f"[MCP] Reading: {key}")
        return self.data.get(key.lower(), 0.0)

    def save_audit_report(self, filename: str, content: str) -> str:
        """MCP Tool: Persists the report to disk."""
        file_path = f"audit_reports/{filename}"
        with open(file_path, "w") as f:
            f.write(content)
        print(f"[MCP] Successfully saved report to: {file_path}")
        return f"Saved to {file_path}"

    def request_human_approval(self, action_description: str) -> bool:
        """
        MCP Tool: Pauses execution to ask a human for approval.
        Useful for high-stakes actions like Slack notifications.
        """
        print(f"\n🛑 [APPROVAL REQUIRED]: {action_description}")
        user_input = input("Proceed? (yes/no): ").strip().lower()
        return user_input == 'yes'

    def notify_slack(self, message: str) -> str:
        """MCP Tool: Sends a notification to the team Slack channel."""
        payload = {"text": f"🚀 *Agent Update:* {message}"}
        print(f"[MCP] Attempting Slack notification...")
        try:
            # Note: This will only succeed if you have a valid webhook URL
            response = requests.post(self.slack_webhook, json=payload, timeout=5)
            if response.status_code == 200:
                return "Slack notification sent successfully."
            return f"Slack returned error: {response.status_code}"
        except Exception as e:
            return f"Failed to send Slack message: {str(e)}"

# --- 2. THE MASTER SKILL (System Logic) ---
MASTER_SKILL = """
# SKILL: End-to-End Audit Specialist
# INSTRUCTIONS:
1. Fetch 'revenue', 'expenses', and 'tax_rate' from the database.
2. Calculate the Net Profit.
3. Save a Markdown report to disk.
4. **HUMAN-IN-THE-LOOP**: You MUST call 'request_human_approval' before sending any Slack notification.
5. If and ONLY IF approved, call 'notify_slack' with the audit summary.
6. If denied, inform the user that the audit is saved but the notification was cancelled.
"""

# --- 3. THE AGENT ORCHESTRATOR ---
class GeminiAgent:
    def __init__(self):
        # Initialize Gemini 3 Client
        self.client = genai.Client(api_key=userdata.get('GEMINI'))
        self.mcp = DatabaseMCPServer()
        self.model_id = "gemini-3-flash-preview"

    def run(self, prompt: str):
        # Configuration with Tools and Thinking Mode
        config = types.GenerateContentConfig(
            system_instruction=MASTER_SKILL,
            tools=[
                self.mcp.fetch_record,
                self.mcp.save_audit_report,
                self.mcp.request_human_approval,
                self.mcp.notify_slack
            ],
            thinking_config=types.ThinkingConfig(
                include_thoughts=True,
                thinking_level="medium" # Corrected case
            ),
            temperature=1.0
        )

        # Start automatic tool-calling session
        chat = self.client.chats.create(model=self.model_id, config=config)
        response = chat.send_message(prompt)

        print("\n" + "="*30)
        for part in response.candidates[0].content.parts:
            if part.thought:
                print(f"🧠 THINKING:\n{part.text}")
            elif part.text:
                print(f"\n✅ AGENT RESPONSE:\n{part.text}")

# --- 4. START THE AGENT ---
agent = GeminiAgent()
agent.run("Perform the Q4 audit and notify the team via Slack.")

[MCP] Reading: revenue
[MCP] Reading: expenses
[MCP] Reading: tax_rate
[MCP] Successfully saved report to: audit_reports/Q4_Audit_Report.md

🛑 [APPROVAL REQUIRED]: Send Q4 Audit Summary to Slack (Net Profit: $16,000)
Proceed? (yes/no): yes
[MCP] Attempting Slack notification...

🧠 THINKING:
**My Reasoning Process**

Okay, so I see there was a problem with the Slack notification - a 404 error. The audit itself, though, that seems to have gone through perfectly fine, and the report is safely stored. My immediate thought is to inform the user. I need to explain the outcome – the good news about the completed audit, and the bad news about the Slack alert failure.

Wait a second. Going back a step… I see the user *did* give their approval. That’s important context. The tool itself is the bit that failed. I need to be careful with my wording here. I want to convey the right status.

Right, the instructions. "If and ONLY IF approved, call 'notify_slack'". Okay, I did that. That's exactly what